In [1]:
import pandas as pd
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [2]:
mask = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[mask, "koi_disposition"] = "False_Positive"
df["koi_disposition"].head()

0         CONFIRMED
1         CONFIRMED
2    False_Positive
3    False_Positive
4         CONFIRMED
Name: koi_disposition, dtype: object

In [3]:
y = df["koi_disposition"]
X = df.drop(columns=["koi_disposition"])
y.head()

0         CONFIRMED
1         CONFIRMED
2    False_Positive
3    False_Positive
4         CONFIRMED
Name: koi_disposition, dtype: object

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\yozga\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [5]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\yozga\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [7]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [8]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [10]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 1s - loss: 0.5571 - acc: 0.7284
Epoch 2/100
 - 1s - loss: 0.3712 - acc: 0.8077
Epoch 3/100
 - 1s - loss: 0.3499 - acc: 0.8248
Epoch 4/100
 - 1s - loss: 0.3525 - acc: 0.8211
Epoch 5/100
 - 1s - loss: 0.3365 - acc: 0.8393
Epoch 6/100
 - 1s - loss: 0.3301 - acc: 0.8451
Epoch 7/100
 - 1s - loss: 0.3304 - acc: 0.8423
Epoch 8/100
 - 1s - loss: 0.3284 - acc: 0.8458
Epoch 9/100
 - 1s - loss: 0.3170 - acc: 0.8544
Epoch 10/100
 - 1s - loss: 0.3109 - acc: 0.8557
Epoch 11/100
 - 1s - loss: 0.3184 - acc: 0.8501
Epoch 12/100
 - 1s - loss: 0.3094 - acc: 0.8576
Epoch 13/100
 - 1s - loss: 0.2999 - acc: 0.8608
Epoch 14/100
 - 1s - loss: 0.2946 - acc: 0.8672
Epoch 15/100
 - 1s - loss: 0.2974 - acc: 0.8675
Epoch 16/100
 - 1s - loss: 0.2949 - acc: 0.8657
Epoch 17/100
 - 1s - loss: 0.2875 - acc: 0.8727
Epoch 18/100
 - 1s - loss: 0.2894 - acc: 0.8739
Epoch 19/100
 - 1s - loss: 0.2837 - acc: 0.8719
Epoch 20/100
 - 1s - loss: 0.2770 - acc: 0.8750
E

In [11]:
model_loss, model_accuracy = model.evaluate(
   X_test_scaled, y_test_categorical, verbose=2)
print(
   f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 0.2457 - acc: 0.8939
Normal Neural Network - Loss: 0.24571573930602497, Accuracy: 0.8938700556755066


In [12]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [13]:
encoded_predictions

array([2, 1, 2, 2, 1, 2, 2, 1, 1, 0], dtype=int64)

In [14]:
prediction_labels

array(['False_Positive', 'CONFIRMED', 'False_Positive', 'False_Positive',
       'CONFIRMED', 'False_Positive', 'False_Positive', 'CONFIRMED',
       'CONFIRMED', 'CANDIDATE'], dtype=object)

In [15]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['False_Positive' 'CONFIRMED' 'False_Positive' 'False_Positive'
 'CONFIRMED' 'False_Positive' 'False_Positive' 'CONFIRMED' 'CONFIRMED'
 'CANDIDATE']
Actual Labels: ['False_Positive', 'CONFIRMED', 'False_Positive', 'False_Positive', 'CONFIRMED', 'False_Positive', 'False_Positive', 'CONFIRMED', 'CONFIRMED', 'CANDIDATE']
